### Call center case - Post-call transcription and analysis with Azure OpenAI Service

In this exercise, we will perform sentiment analysis and summerization using call center transcriptions. We will transribe the customer recording to text, then use OpenAI to detect sentiment. We also use OpenAI to summerize long text into a few sentences for further analysis.

In [11]:
%pip install azure-cognitiveservices-speech

Note: you may need to restart the kernel to use updated packages.


Refer to [Install the Speech SDK
](https://learn.microsoft.com/en-us/azure/ai-services/speech-service/quickstarts/setup-platform?tabs=linux%2Cubuntu%2Cdotnetcli%2Cdotnet%2Cjre%2Cmaven%2Cnodejs%2Cmac%2Cpypi&pivots=programming-language-python). The Speech SDK does not yet support OpenSSL 3.0, which is the default in Ubuntu 22.04 and Debian 12.
To install OpenSSL 1.x from sources on Debian/Ubuntu based systems that don't have it, run the following commands:

```bash
wget -O - https://www.openssl.org/source/openssl-1.1.1u.tar.gz | tar zxf -
cd openssl-1.1.1u
./config --prefix=/usr/local
make -j $(nproc)
sudo make install_sw install_ssldirs
sudo ldconfig -v
export SSL_CERT_DIR=/etc/ssl/certs
```

In [1]:
from dotenv import load_dotenv
from pathlib import Path
import os
from openai import AzureOpenAI

load_dotenv()
os.environ['SSL_CERT_DIR'] = '/etc/ssl/certs'


In [2]:
import os
import time
import azure.cognitiveservices.speech as speechsdk

def recognize_speech_from_file(filename):

    speech_config = speechsdk.SpeechConfig(subscription=os.environ["SPEECH_KEY"], region=os.environ["SPEECH_REGION"])
    speech_config.speech_recognition_language="en-US"
    # speech_config.set_property(speechsdk.PropertyId.Speech_LogFilename, "debug.log")

    # Creates a speech recognizer using a file as audio input, also specify the speech language
    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,  audio_config=audio_config)

    global done 
    done = False
    global recognized_text_list 
    recognized_text_list=[]
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        global done
        done = True

    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)
        # print('RECOGNIZED: {}'.format(evt.result.text))

    # Connect callbacks to the events fired by the speech recognizer
    # speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
    # speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    # speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

    return recognized_text_list

### Sentiment Analysis
Transcribe Customer Call to Text

In [3]:
deployment_name = "gpt-35-turbo-instruct"

# call OpenAI API with model name and prompt
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [4]:
text = recognize_speech_from_file("../data/call_log_1.wav")
print(text)

STT SESSION STARTED: SessionEventArgs(session_id=27e834e52e4b499bbb3cd06e29b56d71)
STT SESSION STOPPED SessionEventArgs(session_id=27e834e52e4b499bbb3cd06e29b56d71)
CLOSING on SessionEventArgs(session_id=27e834e52e4b499bbb3cd06e29b56d71)
['I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.', 'Welcome to find out. It did not and my heater in the garage was running a lot of the time.', 'This one was great and was easily wired.', 'It does turn off when you turn it all the way to the left.', "It's annoying they don't have any in Celsius, but whatever.", 'It runs well and actually turns off my garage heater.', 'Would totally purchase again.', 'Installed with five kilowatts heater in garage.', 'Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


### Create Promot for sentiment analysis
Use natural language to instruct OpenAI to detect customer's sentiment

In [16]:
prompt = f"Detect whether customer is positive or negative.  Just say positive or negative.\n\n{' '.join(text)}"

client.completions.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=deployment_name
).choices[0].text.strip(" \n")

'Negative'

### Use a negative example

In [17]:
text = recognize_speech_from_file("../data/call_log_2.wav")
print(text)
prompt = f"Detect whether customer is positive or negative. Just say positive or negative.\n\n{' '.join(text)}"

client.completions.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=deployment_name
).choices[0].text.strip(" \n")

STT SESSION STARTED: SessionEventArgs(session_id=c2e9d2aaf62e49148a7ad16cf93b3518)
STT SESSION STOPPED SessionEventArgs(session_id=c2e9d2aaf62e49148a7ad16cf93b3518)
CLOSING on SessionEventArgs(session_id=c2e9d2aaf62e49148a7ad16cf93b3518)
['Installed with five kW heater in garage. Thermostat failed in on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


'Negative'

### Summarization

Use OpenAI to summerize customer message

In [18]:
text = recognize_speech_from_file("../data/call_log_1.wav")
print(text)
prompt = f"Summerize the following text.\n\n{' '.join(text)}"

client.completions.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=deployment_name
).choices[0].text.strip(" \n")


STT SESSION STARTED: SessionEventArgs(session_id=b5e8e6e1876b425883c8ebbd51155f3c)
STT SESSION STOPPED SessionEventArgs(session_id=b5e8e6e1876b425883c8ebbd51155f3c)
CLOSING on SessionEventArgs(session_id=b5e8e6e1876b425883c8ebbd51155f3c)
['I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.', 'Welcome to find out. It did not and my heater in the garage was running a lot of the time.', 'This one was great and was easily wired.', 'It does turn off when you turn it all the way to the left.', "It's annoying they don't have any in Celsius, but whatever.", 'It runs well and actually turns off my garage heater.', 'Would totally purchase again.', 'Installed with five kilowatts heater in garage.', 'Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


'The writer purchased a thermostat from a big box store and was assured it would turn off when it got too cold. However, it did not and the heater in the garage ran constantly. They then purchased a different thermostat that was easily wired and did turn off when turned all the way to the left. The writer is annoyed that it does not have a Celsius option but is overall satisfied with its performance. However, there was an incident where the thermostat failed and caused overheated connections, almost resulting in a fire. The installation was done by a certified electrician.'